In [1]:
from aoc_utils import *

In [14]:
import tarfile
fname = 'synacor-challenge.tgz'
if fname.endswith("tar.gz"):
    tar = tarfile.open(fname, "r:gz")
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
elif fname.endswith("tgz"):
    tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

In [ ]:
halt: 0
  stop execution and terminate the program
set: 1 a b
  set register <a> to the value of <b>
push: 2 a
  push <a> onto the stack
pop: 3 a
  remove the top element from the stack and write it into <a>; empty stack = error
eq: 4 a b c
  set <a> to 1 if <b> is equal to <c>; set it to 0 otherwise
gt: 5 a b c
  set <a> to 1 if <b> is greater than <c>; set it to 0 otherwise
jmp: 6 a
  jump to <a>
jt: 7 a b
  if <a> is nonzero, jump to <b>
jf: 8 a b
  if <a> is zero, jump to <b>
add: 9 a b c
  assign into <a> the sum of <b> and <c> (modulo 32768)
mult: 10 a b c
  store into <a> the product of <b> and <c> (modulo 32768)
mod: 11 a b c
  store into <a> the remainder of <b> divided by <c>
and: 12 a b c
  stores into <a> the bitwise and of <b> and <c>
or: 13 a b c
  stores into <a> the bitwise or of <b> and <c>
not: 14 a b
  stores 15-bit bitwise inverse of <b> in <a>
rmem: 15 a b
  read memory at address <b> and write it to <a>
wmem: 16 a b
  write the value from <b> into memory at address <a>
call: 17 a
  write the address of the next instruction to the stack and jump to <a>
ret: 18
  remove the top element from the stack and jump to it; empty stack = halt
out: 19 a
  write the character represented by ascii code <a> to the terminal
in: 20 a
  read a character from the terminal and write its ascii code to <a>; it can be assumed that once input starts, it will continue until a newline is encountered; this means that you can safely read whole lines from the keyboard and trust that they will be fully read
noop: 21
  no operation

In [43]:
import numpy as np
f = open('challenge.bin', 'rb')
ins = np.fromfile(f, dtype=np.uint16)


In [45]:
pnt = 0
ins[pnt]

21

NameError: name 'self' is not defined

In [93]:
class Comp():
    def __init__(self, ins):
        self.pnt = 0
        self.ins = ins
        self.functions = {
            0 : (self.halt,0),
            1 : (self.set,2),
            2 : (self.push,1),
            3 : (self.pop,1),
            4 : (self.eq,3),
            5 : (self.gt,3),
            6 : (self.jmp, 1),
            7 : (self.jt, 2),
            8 : (self.jf, 2),
            9 : (self.add, 3),
            10 : (self.mult, 3),
            19: (self.out,1),
            21: (self.noop,0),
            
        }

    def halt(self, args):
        #should not be called
        sys.exit()

    def set(self, args):
        print('set not implemented')
        sys.exit()

    def push(self, args):
        print('push not implemented')
        sys.exit()

    def pop(self, args):
        print('pop not implemented')
        sys.exit()

    def eq(self, args):
        print('eq not implemented')
        sys.exit()

    def gt(self, args):
        print('gt not implemented')
        sys.exit()

    def jmp(self, args):
        return args[0]

    def jt(self, args):
        print('jt not implemented')
        sys.exit()
        
    def jf(self, args):
        print('jf not implemented')
        sys.exit()
        
    def add(self, args):
        print('add not implemented')
        sys.exit()

    def mult(self, args):
        print('mult not implemented')
        sys.exit()

    def out(self,args):
        print(chr(args[0]), end='')

    def noop(self,args):
        pass


    def do_function(self, pnt):
        # read opcode
        opcode = self.ins[pnt]
        pnt += 1
        func, numargs = self.functions[opcode]
        if func == self.halt:
            return False
        # read arguments
        args = tuple(ins[pnt+i] for i in range(numargs))
        pnt += numargs
        # optional: change pointer
        res = func(args)
        if res: # could do this with walrus
            pnt = res
        # print(opcode, args, func, numargs, pnt)
        return pnt
    
    def run(self,pnt, amount=1000000000000000):
        for i in range(amount):
            pnt = self.do_function(pnt)
            if not pnt:
                print('ending')
                return

    
c = Comp(ins)
c.run(0, 1100)

AttributeError: 'Comp' object has no attribute 'jt'

In [73]:
ins[:10]

array([ 21,  21,  19,  87,  19, 101,  19, 108,  19,  99], dtype=uint16)